# FeatureScaling and Regularization with SciKit-Learn

## Regularization

تا پیش از این ما یک مدل چند جمله ای با بهترین پارامتر را برای این مسئله آماده کردیم اما می توان کمی بهتر از 
این هم عمل کرد

<br>

رگولاریزیشن کمک می کند تا اثر فیچر ها کمتر شود یا به عبارتی مدل ساده تر شود. به بیان ریاضی با تغییر متغیر 
ها ، خروجی کمتر از حالت عادی تغییر کند

<br>

سه روش اصلی برای این کار عبارتند از:
<br>
1.Lasso(l1)
<br>
قدرمطلق فیچر ها
2.Ridge(l2)
<br>
توان 2 فیچر ها
3.ElasticNet
<br>
ترکیبی از لاسو و ریج


## FeatureScaling
فیچر اسکیلینگ هم به نوعی همه داده ها را به شکل استانداردی تبدیل می کند. در بعضی از مدل ها اثر خاصی نمی گذارد مانند بیشتر مدل های درختی، در بعضی از مدل ها باعث افزایش سرعت رسیدن به جواب می شود مانند رگرسیون و در بعضی از مدل ها بودنش خیلی اثر گذاره مانند الگوریتم نزدیک ترین همسایه
<br>
دو نوع مرسوم ترش عبارتند از:
<br>
1.standardization
<br>
به صورت توزیع نرمال با میانگین صفر در می آورد

2.normalization
<br>
همه ی داده ها رو بین صفر و یک قرار می دهد
<br>

**نکته خیلی مهم در فیچر اسکیلینگ: فقط روی داده های ترین فیت می شوند اما ترنسفرم برای هر دو انجام می شود**
<br>
اگر این اتفاق نیفتد
<br>
**DataLeakage**
<br>
اتفاق می افتد! یعنی داده های تست بر روی مدل تاثیرگذار می شوند

## CrossValidation

<br>
K_Fold
<br>
مثلا 10 قسمت تقسیم کنیم و هر سری مدل رو روی بخش تست، تست کنیم و روی بخش ترین، ترین کنیم و این کار روی همه بخش ها انجام میشه در نهایت، میانگین همه خطا ها به عنوان خطای نهایی عنوان میشه
<br>
اینطوری مدل و خطا روی همه داده ها فیت میشن و دیتایی از دست نمیره
<br>
میتونیم یه قسمتی از داده ها رو
<br>
Hold_Out_data
<br>
نگه داریم و در نهایت بعد از مشخص شدن پارامتر های نهایی، خطای نهایی رو روی اون ها اندازه گیری کنیم
<br>

**یکی از کاربردهای اصلی این روش برای زمان هایی هست که می خوایم چندین پارامتر رو تست کنیم و به بهترین اون برسیم، مانند رگولاریزیشن و پارامتر های مختلف آلفا**




## فراخوانی کتابخانه ها

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## فراخوانی دیتا و تعیین ورودی و خروجی

In [2]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

### تغییر شکل متغیر های ورودی به شکل چند جمله ای با درجه 3

In [3]:
from sklearn.preprocessing import PolynomialFeatures

In [4]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [5]:
poly_features = polynomial_converter.fit_transform(X)

### Train | Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

----
----

## Scaling the Data

به طور کلی انجام این کار ضرری برای هیچ مدلی نداره! پس توصیه می شه همیشه این کار رو انجام بدین

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# help(StandardScaler)

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
X_train = scaler.transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

## Ridge Regression


In [15]:
from sklearn.linear_model import Ridge

آلفا همون لاندا در رگولاریزیشن هست


In [16]:
ridge_model = Ridge(alpha=10)

In [17]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [18]:
test_predictions = ridge_model.predict(X_test)

In [19]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [20]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [21]:
MAE

0.5774404204714173

In [22]:
RMSE

0.8946386461319678

How did it perform on the training set? (This will be used later on for comparison)

In [23]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.5288348183025328

### انتخاب پارامتر الفا از طریق کراس ولیدیشن


کراس ولیدیشن به معنی اینه که کل داده ها رو به کی فولد
<br>
K_Fold
<br>
مثلا 10 قسمت تقسیم کنیم و هر سری مدل رو روی بخش تست، تست کنیم و روی بخش ترین، ترین کنیم و این کار روی همه بخش ها انجام میشه در نهایت، میانگین همه خطا ها به عنوان خطای نهایی عنوان میشه
<br>
اینطوری مدل و خطا روی همه داده ها فیت میشن و دیتایی از دست نمیره
<br>
میتونیم یه قسمتی از داده ها رو
<br>
Hold_Out_data
<br>
نگه داریم و در نهایت بعد از مشخص شدن پارامتر های نهایی، خطای نهایی رو روی اون ها اندازه گیری کنیم
<br>
**یکی از کاربردهای اصلی این روش برای زمان هایی هست که می خوایم چندین پارامتر رو تست کنیم و به بهترین اون برسیم، مانند رگولاریزیشن و پارامتر های مختلف آلفا**

In [24]:
from sklearn.linear_model import RidgeCV

In [28]:
# help(RidgeCV)

In [26]:
# scoring:
# روشی هست که در کراس ولیدیشن میانگین خطا ها رو با اون می گیره
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [27]:
# هر چی تعداد آلفا ها بیشتر باشه، ترین کردن مدل زمان بر تر خواهد بود
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
        gcv_mode=None, normalize=False, scoring='neg_mean_absolute_error',
        store_cv_values=False)

In [28]:
# مشاهده بهترین مقدار آلفا
ridge_cv_model.alpha_

0.1

In [29]:
test_predictions = ridge_cv_model.predict(X_test)

In [30]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [31]:
MAE

0.42737748843249934

In [32]:
RMSE

0.6180719926923581

In [33]:
# Training Set Performance
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.3094132105641705

In [34]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])


-----

## Lasso Regression

In [35]:
from sklearn.linear_model import LassoCV

In [37]:
#می تونیم الفا های مختلف رو تعیین کنیم یا بگیم خودش 100 تا الفا رو در یک بازه ای در نظر بگیره
# پارامتر سوم همون تعداد فولد ها در کراس ولیدیشن هست

lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [38]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.1, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [39]:
lasso_cv_model.alpha_

0.49430709092258285

In [40]:
test_predictions = lasso_cv_model.predict(X_test)

In [41]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [42]:
MAE

0.6541723161252858

In [43]:
RMSE

1.1308001022762542

In [44]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.69128071408207

همونطور که مشاهده میشه لاسو خیلی از فیچر ها رو بی اثر می کنه و این باعث میشه سرعت حل مسئله خیلی بیشتر بشه




In [45]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

## Elastic Net

ترکیبی از دو روش ریج و لاسو هست

In [46]:
from sklearn.linear_model import ElasticNetCV

پارامتر اول درصد مدل از لاسو بودن رو تعیین می کنه که زمانی که برابر با یک باشه، انگار 100 درصد لاسو هست

In [47]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],eps=0.1,n_alphas=100)

In [48]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.1, fit_intercept=True,
             l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000,
             n_alphas=100, n_jobs=None, normalize=False, positive=False,
             precompute='auto', random_state=None, selection='cyclic',
             tol=0.0001, verbose=0)

In [49]:
elastic_model.l1_ratio_

1.0

بهترین پارامتر 1 تعیین شد یعنی مدل کاملا شبیه لاسو خواهد بود

In [50]:
test_predictions = elastic_model.predict(X_test)

In [51]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [52]:
MAE

0.6541723161252858

In [53]:
RMSE

1.1308001022762542

In [54]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

0.69128071408207

In [55]:
elastic_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

**یه نکته، می تونیم از اول از روش الاستیک استفاده کنیم و پارامتر ها رو جوری تعیین کنیم که با هم بتونه 100 درصد لاسو شه و هم 100 در صد ریج**
